In [5]:
import warnings
warnings.filterwarnings("ignore")

import unyt as u

from flowermd.base import Pack, Simulation
from flowermd.library.polymers import PPS, LJChain, EllipsoidChain
from flowermd.library.forcefields import BeadSpring, KremerGrestBeadSpring, EllipsoidForcefield, OPLS_AA_PPS

In [2]:
pps_chains = PPS(num_mols=25, lengths=20)
bead_spring_chains = LJChain(num_mols=25, lengths=20)
ellipsoid_chains = EllipsoidChain(num_mols=25, lengths=20, lpar=1, bead_mass=1.0, bond_length=0.1)

In [3]:
pps_system = Pack(molecules=pps_chains, density=1.2 * u.g/(u.cm**3))
bead_spring_system = Pack(molecules=bead_spring_chains, density=0.6/u.nm**3)
ellipsoid_system = Pack(molecules=ellipsoid_chains, density=0.6/u.nm**3)

In [4]:
bead_spring_ff = BeadSpring(
    beads={"A": dict(epsilon=1.0, sigma=1.0)},
    bonds={"A-A": dict(k=300, r0=1.1)},
    angles={"A-A-A": dict(k=300, t0=2.6)},
    r_cut=2.5
)

ellipsoid_ff = EllipsoidForcefield(
    lpar=1.0,
    lperp=0.4,
    epsilon=1.0,
    r_cut=2.5,
    bond_k=300,
    bond_r0=0.1,
    angle_k=150,
    angle_theta0=2.6
)

pps_system.apply_forcefield(force_field=OPLS_AA_PPS(), r_cut=2.5, auto_scale=True)

In [9]:
pps_sim = Simulation(initial_state=pps_system.hoomd_snapshot, forcefield=pps_system.hoomd_forcefield)
bead_spring_sim = Simulation(initial_state=bead_spring_system.hoomd_snapshot, forcefield=bead_spring_ff.hoomd_forces)
ellipsoid_sim = Simulation(initial_state=ellipsoid_system.hoomd_snapshot, forcefield=ellipsoid_ff.hoomd_forces)

Initializing simulation state from a gsd.hoomd.Frame.
Initializing simulation state from a gsd.hoomd.Frame.
Initializing simulation state from a gsd.hoomd.Frame.
